# **NAMED ENTITY RECOGNITION (NER)**

In [ ]:
%pip install -q -r requirements.txt

In [1]:
import numpy as np
import spacy
import nltk
import svgling
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

In [ ]:
# Function definitions

In [2]:
nltk.download('conll2002')
from nltk.corpus import conll2002

# Spanish
train_esp = conll2002.iob_sents('esp.train') # Train
val_esp = conll2002.iob_sents('esp.testa') # Val
test_esp = conll2002.iob_sents('esp.testb') # Test
# Dutch
train_ned = conll2002.iob_sents('ned.train') # Train
val_ned = conll2002.iob_sents('ned.testa') # Val
test_ned = conll2002.iob_sents('ned.testb') # Test

[nltk_data] Downloading package conll2002 to C:\Users\Cai Selvas
[nltk_data]     Sala\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\conll2002.zip.


In [3]:
from class_feature_getter import FeatureGetter

feature_getter = FeatureGetter()

In [ ]:
tagger_esp = nltk.tag.CRFTagger(feature_func=feature_getter)
tagger_ned = nltk.tag.CRFTagger(feature_func=feature_getter)

In [ ]:
words_esp = nltk.word_tokenize('spanish')
words_ned = nltk.word_tokenize('dutch')

In [ ]:
tagged_text_esp = tagger_esp.tag(words_esp)
tagged_text_ned = tagger_ned.tag(words_ned)

In [ ]:
tagger_esp.train(train_esp, model_file='./models/model_esp')
tagger_ned.train(train_ned, model_file='./models/model_ned')